1. Load and check data　　-データの読み込み-
 * 1.1 Load data (train, test, submit)　-データの読み込み-
 * 1.2 Show outline of train data　-訓練データの概要の表示-
 * 1.3 Divide into train and target　-教師データの抽出-
 * 1.4 Concatenate train and test　-訓練データとテストデータの結合-<br><br>
2. Analyze feature of data　　-データの特徴を可視化-
 * 2.1 Plot categorical data　　-カテゴリデータの可視化-
    * Sex
    * Pclass
    * Embarked<br><br>
 * 2.2 Plot numerical data　　-数値データの可視化-
    * SibSp
    * Parch
    * Age
    * Fare<br><br>
3. Fill Nah data　-欠損値の補充-
 * 3.1 Number of missing data and correlation for each column　-カラムごとの欠損値と、相関関係- <br><br>
 * 3.2 Fill missing Data Considering correlation　-相関関係を考慮した欠損値の補充-
    * Fare
    * Age
    * Embarked<br><br>
4. Feature engineering　　-特徴量エンジニアリング-
 * 4.1 Create Family features　-新たに家族数を特徴量として作成する-
    * FamilySize
    * Ticket<br><br>

5. Delete unused data　　-カラムの整理-
 * 5.1 Delete unused column -学習に使わないカラムを除外する-
 * 5.2 Align data　types -学習用にデータ型を揃える-<br><br>

6. Make ML Model　　-機械学習モデルの構築-
 * 6.1 Import packages to need -必要なパッケージをインポート-
 * 6.2 Divide alldata into train data and pred data　-結合したデータを学習、予測用データに分割する-
 * 6.3 Divide train data into train_data and test_data -学習用データを学習、検証用データに分割する-
 * 6.4 Convert dataset for LightGBM　-LightGBM用にデータを変換する-
 * 6.5 Build LightGBM model　-LightGBMモデルを作成する-
 * 6.6 Predict with model -予測を行う-<br><br>
7. Submit prediction -予測データの提出-


In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import seaborn as sns

## 1. Load and check data　-データの読み込み-

### 1.1 Load data (train, test, submit)　-データの読み込み-

In [ ]:
train_data = pd.read_csv('../input/titanic/train.csv')
test_data = pd.read_csv('../input/titanic/test.csv')
gender_submission_data = pd.read_csv('../input/titanic/gender_submission.csv')

### 1.2 Show outline of train data　-訓練データの概要の表示-

In [ ]:
train_data.head()

### 1.3 Divide into train and test　-教師データの抽出-

In [ ]:
# create target data 'Survived' -Survivedカラム(教師データ)の作成-
target = train_data['Survived']

# drop columns 'Survived' from train data -訓練データからSurvivedカラムを削除-
train = train_data.drop('Survived', axis=1)


### 1.4 Concatenate train and test data　-訓練データとテストデータの結合-

In [ ]:
# concatenate train and test data　-訓練データとテストデータの結合-
all_data = pd.concat([train, test_data]).reset_index(drop=True)

print(train.shape)
print(test_data.shape)
print(all_data.shape)

In [ ]:
# show outline of data -データの概要-
all_data.info()

### 2.Analyze feature of data　-データの特徴を可視化-

#### 2.1 Plot categorical data　-カテゴリデータの可視化-

#### Sex

In [ ]:
sns.barplot(x=train['Sex'], y=target)

Women have higher survival probability than men　-女性の方が生存率が高い事が分かる-<br>
'Sex' is an important factor in prediction -性別は予測に対して重要な要素である事が分かる-

#### Pclass

In [ ]:
sns.barplot(x=train['Pclass'], y=target)

Pclass (1st,2nd,3rd)
1st class has higher chance to survive than 2nd,3rd -ファーストクラスはセカンド、サードクラスより生存率が高いことが分かる-

In [ ]:
sns.barplot(x=train['Pclass'], y=target, hue=train['Sex'])

#### Embarked

In [ ]:
sns.barplot(x=train['Embarked'], y=target)

'Embarked C' has higher chance to survive than 'S','Q'　-乗船場'C'の人は'S','Q'よりも生存率が高い事がわかる-

#### 2.2 Plot numerical data

In [ ]:
# select numeric column from train_data　-数値データのカラムを列挙する-
train_data_int = ['Survived', 'Age', 'SibSp', 'Parch', 'Fare']
# plot correlation matrix between numerical data　-数値データ間の相関係数を表示する-
sns.heatmap(train_data[train_data_int].corr(), annot=True)

It seems that 'Fare' has important correlation with 'Survived'　-運賃が生存率に対して高い相関を示している事がわかる-<br>
But other columns excepting 'Fare' have to be researched　-運賃以外のカラムに関しても調査しておく-

### SibSp

In [ ]:
sns.barplot(x=train['SibSp'], y=target)

the passenger who has many 'Sibsb(siblings/spouses)' has lower chance to survive.　-兄弟、配偶者が３人以上であると生存率が低くなる傾向がわかる。-

### Parch

In [ ]:
sns.barplot(x=train['Parch'], y=target)

small family has higer chace to survive than single(Parch=0) and medium(Parch=4,5,...)　-両親や子供が合計1~3人の乗客が生存率が高い事が分かる。-

### Age

In [ ]:
age_glaph = sns.FacetGrid(train_data, 'Survived')
age_glaph.map(sns.distplot, 'Age')

In [ ]:
age_glaph = sns.kdeplot(train_data['Age'][(train_data['Survived'] == 0) & (train_data['Age'].notnull())], color='Red', shade=True)
age_glaph = sns.kdeplot(train_data['Age'][(train_data['Survived'] == 1) & (train_data['Age'].notnull())], color='Blue', shade=True)
age_glaph.legend(['Not Survived', 'Survived'])

seeing age between 0 ~ 10, it seems that very younger passenger has higher chance to survive than older　-0~10歳の子供は優先的に救助されたと推測できる。-

### Fare

In [ ]:
sns.distplot(all_data['Fare'])

as we can see, abount 'Fare' data is very skewed, so transform it with 'log' to make shape like normal distribution　-運賃データが非常に偏りがあるためlogを使用し正規分布に近づける-

In [ ]:
all_data['Fare'] = np.round(np.log(all_data['Fare'][all_data['Fare'] > 0]),5)

In [ ]:
sns.distplot(all_data['Fare'])

## 3.Fill missing data　-欠損値の補充-

### 3.1 Number of missing data and correlation for each column　-カラムごとの欠損値と、相関関係-

In [ ]:
all_data.isnull().sum()

In [ ]:
sns.heatmap(train_data.corr(), annot=True)

### 3.2 Fill missing Data Considering correlation　-相関関係を考慮した欠損値の補充-

### Fare

'Fare' has strong correlation with 'Pclass'　-運賃は'Pclass'に強い相関関係が見える-

In [ ]:
# fill missing 'Fare' data with average value for each 'Pclass'　-'Pclass'ごとの料金の平均値を欠損値に補充する-
all_data['Fare'] = all_data.groupby('Pclass')['Fare'].transform(lambda x :x.fillna(x.mean()))
print(all_data['Fare'].isnull().sum())

### Age

'Age' has strong correlation with 'Pclass', 'SibSp', 'Parch'　-年齢は['Pclass','SibSp','Parch']に対して強い相関がある事が分かる-

In [ ]:
fare_nan_indx = all_data['Age'][all_data['Age'].isnull()].index.tolist()

age_median = all_data['Age'].median()
for idx in fare_nan_indx:
    age_pred_median = all_data['Age'][(all_data['Pclass'] == all_data.iloc[idx]['Pclass']) & (all_data['SibSp'] == all_data.iloc[idx]['SibSp']) & (all_data['Parch'] == all_data.iloc[idx]['Parch'])].median()
    if not np.isnan(age_pred_median):
        all_data['Age'].iloc[idx] = age_pred_median
    else:
        all_data['Age'].iloc[idx] = age_median
    
print(all_data['Age'].isnull().sum())

### Embarked

In [ ]:
all_data['Embarked'].fillna('S')
all_data = pd.get_dummies(all_data, columns=['Embarked'])

## 4.feature engineering

### 4.1 Create Family features　-新たに家族数を特徴量として作成する-

#### Family size

In [ ]:
all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1

In [ ]:
sns.barplot(x=all_data['FamilySize'], y=target)

familySize=4 has highest chance to survive. and survival probability is worst for larger one　-'familySize'が４の時最も生存率が高いことを示している。４を越えた途端、生存率が下がり始めている-

#### Ticket

Tickets with same prefixes may have a similar class and survival.

In [ ]:
all_data['Ticket'].isnull().sum()

In [ ]:
Tickets = []
for t in all_data['Ticket'].tolist():
    if not t.isdigit():
        Tickets.append(t.replace('/','').replace('.','').split(' ')[0])
    else:
        Tickets.append('X')
all_data['Ticket'] = Tickets
all_data['Ticket'].head()

In [ ]:
plt.figure(figsize=(35,35))
sns.barplot(x=all_data['Ticket'], y=target)

In [ ]:
all_data = pd.get_dummies(all_data,columns=['Ticket'],prefix='T')

## 5.Delete unused data　-カラムの整理-

### 5.1 Delete unused column -学習に使わないカラムを除外する-


In [ ]:
all_data.drop(['PassengerId', 'Name', 'Cabin'],axis=1, inplace=True)

In [ ]:
all_data = pd.get_dummies(all_data, columns=['Sex'])

### 5.2 Align data　types -学習用にデータ型を揃える-

In [ ]:
all_data[all_data.dtypes[all_data.dtypes == 'uint8'].index.tolist()] = all_data[all_data.dtypes[all_data.dtypes == 'uint8'].index.tolist()].astype(float)
all_data.dtypes


## 6.Make ML Model


### 6.1 Import packages to need -必要なパッケージをインポート-

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
# from sklearn.model_selection import KFold


### 6.2 Divide alldata into train data and pred data　-結合したデータを学習、予測用データに分割する-

In [ ]:
train_data_for_model = all_data[:train_data.shape[0]]
pred_data_for_model = all_data[train_data.shape[0]:]

### 6.3 Divide train data into train_data and test_data -学習用データを学習、検証用データに分割する-

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_data_for_model, target, test_size=0.005)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


### 6.4 Convert dataset for LightGBM　-LightGBM用にデータを変換する-

In [ ]:
lgb_train = lgb.Dataset(np.array(train_x),np.array(train_y))
lgb_eval = lgb.Dataset(np.array(test_x),np.array(test_y), reference=lgb_train)

### 6.5 Build LightGBM model　-LightGBMモデルを作成する-

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metrics': {'l2'},
    'learning_rate': 0.003,
    'num_iterations':100,
    'feature_fraction':0.52,
    'bagging_fraction':0.79,
    'verbose':0
}

gbm = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=1000)

### 6.6 Predict with model -予測を行う-

In [ ]:
pred = gbm.predict(np.array(test_data_for_model), num_iteration=gbm.best_iteration)
pred

## 7. Submit Prediction　-予測データの提出-

In [ ]:
pred = np.where(pred > 0.42,1,0)
submit = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': pred})
submit.to_csv('submit.csv',index=False)